In [1]:
##########################################################################
# Created on Sat Nov 21 22:14:45 2021                                    #
# Python for Financial Analysis and Risk Management                      #
# @author: Meng Lipeng (FRM, CFA)                                        #
##########################################################################

# 9.2.2.Cashflow

**Fixed payer, floating receiver**
$$f_i=\frac{1}{m}(R_{i-1}-R)L\tag{9-1}$$
**Fixed receiver, floating payer**
$$f_i=\frac{1}{m}(R-R_{i-1})L\tag{9-2}$$
where,\
$L$:Notional amount\
$m$:Payment frequency per year\
$R_{i-1}$:Floating rate determined at $t_{i-1}$ and payed at $t_i$\
$R$:Fixed rate which is also called swap rate

In [2]:
def IRS_cashflow(R_flt,R_fix,L,m,position):
    '''Define a function to calculate cashflow of IRS.
    R_flt:floating rate, input as array
    R_fix:fixed rate
    L:notional amount
    m:payment frequency per year
    position:'long' indicates fixed payer, otherwise fixed receiver'''
    if position=='long':
        cashflow=(R_flt-R_fix)*L/m
    else:
        cashflow=(R_fix-R_flt)*L/m
    return cashflow

In [3]:
import numpy as np

rate_float=np.array([0.031970,0.032000,0.029823,0.030771,0.044510,0.047093,0.043040,0.032750,0.029630,0.015660])
rate_fixed=0.037
par=1e8
M=2

Netpay_A=IRS_cashflow(R_flt=rate_float,R_fix=rate_fixed,L=par,m=M,position='long')
Netpay_A

array([ -251500.,  -250000.,  -358850.,  -311450.,   375500.,   504650.,
         302000.,  -212500.,  -368500., -1067000.])

In [4]:
Netpay_B=IRS_cashflow(R_flt=rate_float,R_fix=rate_fixed,L=par,m=M,position='short')
Netpay_B

array([ 251500.,  250000.,  358850.,  311450., -375500., -504650.,
       -302000.,  212500.,  368500., 1067000.])

In [5]:
Totalpay_A=np.sum(Netpay_A)
Totalpay_B=np.sum(Netpay_B)
print('Total amount payed by A is ',round(Totalpay_A,2))
print('Total amount payed by B is ',round(Totalpay_B,2))

Total amount payed by A is  -1637650.0
Total amount payed by B is  1637650.0


# 9.2.3.Equivalence

**Fixed payer, floating receiver**
$$V_{IRS}=B_{flt}-B_{fix}\tag{9-3}$$
**Fixed receiver, floating payer**
$$V_{IRS}=B_{fix}-B_{flt}\tag{9-4}$$
When contract initial,
$$B_{fix}=B_{flt}\tag{9-5}$$

# 9.2.4.Pricing swap rate

$$B_{fix}=\left(\frac{R}{m}\sum_{i=1}^{N}e^{-y_it_i}+e^{-y_NT}\right)L\tag{9-6}$$
where,\
R:Fixed rate\
m:Payment frequency per year(m>=1)\
L:Notional amount\
$y_i$:Discount rate at $t_i$(continuous compound)\
$t_i$:Time to payment date(i=1,2,....,N)\
$T$:Contract maturity

$$B_{fix}=\left(\frac{R}{m}\sum_{i=1}^{N}q_i+q_N\right)L\tag{9-7}$$
where,\
$q_i=e^{-y_it_i}$:discount factor at $t_i$

$$B_{flt}=L\tag{9-8}$$

Combine (9-5),(9-7) and (9-8),
$$R=\frac{m(1-q_N)}{\sum_{i=1}^Nq_i}\tag{9-9}$$

In [6]:
def swap_rate(m,y,T):
    '''Define a function to calculate swap rate.
    m:payment frequency per year
    y:continuous compound zero rate(discount rate), input as array
    T:contract tenor in year'''
    import numpy as np
    n_list=np.arange(1,m*T+1)
    t=n_list/m
    q=np.exp(-y*t)
    rate=m*(1-q[-1])/np.sum(q)
    return rate

In [7]:
freq=2
tenor=3

r_list=np.array([0.020579,0.021276,0.022080,0.022853,0.023527,0.024036])
R_July1=swap_rate(m=freq,y=r_list,T=tenor)
print('Swap rate of contract is ',round(R_July1,4))

Swap rate of contract is  0.0241


# 9.2.5.Valuation

$$B_{fix}=\left(\frac{R_{fix}}{m}\sum_{i=1}^{N}e^{-y_i\tilde{t}_i}+e^{-y_N\tilde{T}}\right)L\tag{9-10}$$
where,\
$\tilde{t}_i$:valuation date to next ith payment date\
$\tilde{T}$:contract remaining tenor in year

$$B_{flt}=\left(\frac{R_{flt}}{m}+1\right)Le^{-y_1\tilde{t}_1}\tag{9-11}$$

In [14]:
def swap_value(R_fix,R_flt,t,y,m,L,position):
    '''Define a function for IRS valuation.
    R_fix:swap rate
    R_flt:next payment floating rate
    t:valution date to next ith payment date, input as array
    y:continuous compound zero rate(discount rate), input as array
    m:payment frequency per year
    L:notional amount
    position:'long' indicates fixed payer, otherwise fixed receiver'''
    from numpy import exp
    B_fix=(R_fix*sum(exp(-y*t))/m+exp(-y[-1]*t[-1]))*L
    B_flt=(R_flt/m+1)*L*exp(-y[0]*t[0])
    if position=='long':
        value=B_flt-B_fix
    else:
        value=B_fix-B_flt
    return value

In [9]:
import scipy.interpolate as si

T=np.array([1/12,2/12,0.25,0.5,0.75,1.0,2.0,3.0])
R_July10=np.array([0.017219,0.017526,0.021012,0.021100,0.021764,0.022165,0.025040,0.026894])
R_July20=np.array([0.016730,0.018373,0.019934,0.020439,0.021621,0.022540,0.024251,0.025256])

func_July10=si.interp1d(x=T,y=R_July10,kind='cubic')
func_July20=si.interp1d(x=T,y=R_July20,kind='cubic')

T_new=np.array([1/12,2/12,0.25,0.5,0.75,1.0,1.5,2.0,2.5,3.0])

R_new_July10=func_July10(T_new)
R_new_July10

array([0.017219  , 0.017526  , 0.021012  , 0.0211    , 0.021764  ,
       0.022165  , 0.0231695 , 0.02504   , 0.02665525, 0.026894  ])

In [10]:
R_new_July20=func_July20(T_new)
R_new_July20

array([0.01673   , 0.018373  , 0.019934  , 0.020439  , 0.021621  ,
       0.02254   , 0.02357906, 0.024251  , 0.02474644, 0.025256  ])

In [11]:
import datetime as dt

T1=dt.datetime(2020,7,10)
T2=dt.datetime(2020,7,20)
T3=dt.datetime(2021,1,1) #next payment date

tenor1=(T3-T1).days/365
tenor2=(T3-T2).days/365

T=3
M=2

T_list1=np.arange(T*M)/M
T_list1=T_list1+tenor1
T_list1

array([0.47945205, 0.97945205, 1.47945205, 1.97945205, 2.47945205,
       2.97945205])

In [12]:
T_list2=np.arange(T*M)/M
T_list2=T_list2+tenor2
T_list2

array([0.45205479, 0.95205479, 1.45205479, 1.95205479, 2.45205479,
       2.95205479])

In [15]:
yield_July10=np.zeros_like(T_list1)
yield_July10[0]=R_new_July10[3]
yield_July10[1:]=R_new_July10[5:]

yield_July20=np.zeros_like(T_list2)
yield_July20[0]=R_new_July20[3]
yield_July20[1:]=R_new_July20[5:]

rate_fix=0.0241
rate_float=0.02178
par=1e8

value_July10_long=swap_value(R_fix=rate_fix,R_flt=rate_float,t=T_list1,y=yield_July10,m=M,L=par,position='long')
value_July10_short=swap_value(R_fix=rate_fix,R_flt=rate_float,t=T_list1,y=yield_July10,m=M,L=par,position='short')
print('IRS value for long position at 2020/07/10 is ',round(value_July10_long,2))
print('IRS value for short position at 2020/07/10 is ',round(value_July10_short,2))

IRS value for long position at 2020/07/10 is  848538.61
IRS value for short position at 2020/07/10 is  -848538.61


In [16]:
value_July20_long=swap_value(R_fix=rate_fix,R_flt=rate_float,t=T_list2,y=yield_July20,m=M,L=par,position='long')
value_July20_short=swap_value(R_fix=rate_fix,R_flt=rate_float,t=T_list2,y=yield_July20,m=M,L=par,position='short')
print('IRS value for long position at 2020/07/20 is ',round(value_July20_long,2))
print('IRS value for short position at 2020/07/20 is ',round(value_July20_short,2))

IRS value for long position at 2020/07/20 is  404294.41
IRS value for short position at 2020/07/20 is  -404294.41
